In [29]:
import os

if os.path.exists("bench") == False:
    os.mkdir("bench")

!wget https://arxiv.org/pdf/1706.03762 -O bench/attention.pdf
!wget https://arxiv.org/pdf/2407.01449 -O bench/colpali.pdf
!wget https://arxiv.org/pdf/2310.06825 -O bench/mistral.pdf


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2024-10-10 16:32:48--  https://arxiv.org/pdf/1706.03762
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.3.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2215244 (2.1M) [application/pdf]
Saving to: ‘bench/attention.pdf’

bench/attention.pdf 100%[===================>]   2.11M  5.13MB/s    in 0.4s    

2024-10-10 16:32:48 (5.13 MB/s) - ‘bench/attention.pdf’ saved [2215244/2215244]



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2024-10-10 16:32:49--  https://arxiv.org/pdf/2407.01449
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.195.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9380119 (8.9M) [application/pdf]
Saving to: ‘bench/colpali.pdf’

bench/colpali.pdf   100%[===================>]   8.95M  6.08MB/s    in 1.5s    

2024-10-10 16:32:50 (6.08 MB/s) - ‘bench/colpali.pdf’ saved [9380119/9380119]

--2024-10-10 16:32:50--  https://arxiv.org/pdf/2310.06825
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.67.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


HTTP request sent, awaiting response... 200 OK
Length: 3749788 (3.6M) [application/pdf]
Saving to: ‘bench/mistral.pdf’

bench/mistral.pdf   100%[===================>]   3.58M  4.32MB/s    in 0.8s    

2024-10-10 16:32:51 (4.32 MB/s) - ‘bench/mistral.pdf’ saved [3749788/3749788]



In [30]:
# from sentence_transformers import SentenceTransformer
# import pymupdf
# from semantic_text_splitter import TextSplitter
# import time
# splitter = TextSplitter(1000)
# files = []
# model = SentenceTransformer('BAAI/bge-small-en-v1.5')

# # get all pdfs from test_files
# now = time.time()

# for file in os.listdir("bench"):

#   text = []
#   doc = pymupdf.open("bench/" + file)

#   for page in doc:
#       text.append(page.get_text())

#   text = " ".join(text)
#   chunks = splitter.chunks(text)
#   embeddings = model.encode(chunks)


# print(time.time() - now)



In [31]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [10]:
from fastembed import TextEmbedding
import pymupdf
from semantic_text_splitter import TextSplitter
import time
import os
import numpy as np
splitter = TextSplitter(1000)
files = []
model = TextEmbedding(model_name = "BAAI/bge-small-en-v1.5",
                      providers=["CUDAExecutionProvider", "CPUExecutionProvider"])

# get all pdfs from test_files
now = time.time()

for file in os.listdir("bench"):

  text = []
  print(file)
  doc = pymupdf.open("bench/" + file)

  for page in doc:
      text.append(page.get_text())

  text = " ".join(text)
  chunks = splitter.chunks(text)
  embeddings = list(model.embed(chunks))
  embeddings = np.array(embeddings)
  print(embeddings.shape)
print(time.time() - now)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

colpali.pdf
(68, 384)
mistral.pdf
(26, 384)
attention.pdf
(42, 384)
0.36588358879089355


In [16]:
model.list_supported_models()

[{'model': 'BAAI/bge-base-en',
  'dim': 768,
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year',
  'size_in_GB': 0.42,
  'sources': {'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz'},
  'model_file': 'model_optimized.onnx'},
 {'model': 'BAAI/bge-base-en-v1.5',
  'dim': 768,
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year',
  'size_in_GB': 0.21,
  'sources': {'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   'hf': 'qdrant/bge-base-en-v1.5-onnx-q'},
  'model_file': 'model_optimized.onnx'},
 {'model': 'BAAI/bge-large-en-v1.5',
  'dim': 1024,
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year',
  'size_in_GB': 1.2,
  'sources': {'

In [8]:
from embed_anything import EmbeddingModel, WhichModel, TextEmbedConfig, embed_query, embed_directory
import time

import pymupdf
from semantic_text_splitter import TextSplitter
import time
import os

config = TextEmbedConfig(256,4, buffer_size = 64)
model = EmbeddingModel.from_pretrained_onnx(WhichModel.Bert, "BAAI/bge-small-en-v1.5")
splitter = TextSplitter(1000)
files = []


# get all pdfs from test_files
now = time.time()

for file in os.listdir("bench"):
  text = []
  print(file)
  doc = pymupdf.open("bench/" + file)

  for page in doc:
      text.append(page.get_text())
  
  text = " ".join(text)
  chunks = splitter.chunks(text)
  embeddings = embed_query(chunks, model, config)
print(time.time() - now)


colpali.pdf
mistral.pdf
attention.pdf
0.7806596755981445
